In [37]:
import json
import warnings
import pandas as pd
pd.options.mode.chained_assignment = None
from sec_api import QueryApi, XbrlApi

In [80]:
api_key = "e3d46df2271d81a4b7a71153aef74fa5e7dae956314122b155767267425b36a6"

# Query 20 most recent 10Q Info

In [ ]:
# queryAPI = QueryApi(api_key=api_key)

In [ ]:
# query = {"query": {
#             "query_string": {
#                 "query": "ticker:TSLA AND formType:\"10-Q\"",
#                     }
#                 },
#          "from": "0",
#          "size": 4*5,
#          "sort": [{"filedAt": {"order": "desc"}}]
#             }

In [ ]:
# response = queryAPI.get_filings(query)
# response_parsed = pd.DataFrame.from_records(response["filings"])

In [74]:
# response_parsed.to_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv", index=False)
response_parsed = pd.read_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv")

# Parse 10Q

In [81]:
# xbrlAPI = XbrlApi(api_key)

In [94]:
# %%time
# for date, url in response_parsed[["periodOfReport", "linkToFilingDetails"]].values:
#     xbrl_json = xbrlAPI.xbrl_to_json(url)
    
#     with open(f"tesla_10Q_{date}.json", "w") as f:
#         json.dump(xbrl_json, f)

CPU times: total: 4.41 s
Wall time: 10.9 s


## Extract Data

In [95]:
with open("tesla_10Q_2023-06-30.json", "r") as f:
    xbrl_json = json.load(f)

In [96]:
xbrl_json["CoverPage"]["DocumentPeriodEndDate"]

'2023-06-30'

In [97]:
BalanceSheets = xbrl_json["BalanceSheets"]
StatementsOfIncome = xbrl_json["StatementsOfIncome"]
StatementsOfCashFlows = xbrl_json["StatementsOfCashFlows"]
StatementsOfShareholdersEquity = xbrl_json["StatementsOfShareholdersEquity"]

## Balance Sheets

In [98]:
def balancesheets_to_df(BalanceSheets):
    """
    """
    df_list = []

    for name in BalanceSheets.keys():
        try:
            df = pd.DataFrame.from_records(BalanceSheets[name])
            df["period"] = df["period"].apply(lambda x: x["instant"])
            df["name"] = name
            df = df[["name", "period", "value"]]
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [99]:
set(BalanceSheets.keys()).difference(set(balancesheets_to_df(BalanceSheets).name))

CommitmentsAndContingencies --- "['value'] not in index"


{'CommitmentsAndContingencies'}

## Statements of Income

In [100]:
def incomestatement_to_df(StatementsOfIncome):
    """
    """
    df_list = []

    for name in StatementsOfIncome.keys():
        try:
            df = pd.DataFrame.from_records(StatementsOfIncome[name])
            df["startDate"] = df["period"].apply(lambda x: x["startDate"])
            df["endDate"] = df["period"].apply(lambda x: x["endDate"])
            df["name"] = name
            df = df[["name", "startDate", "endDate", "value"]]
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [101]:
set(StatementsOfIncome.keys()).difference(set(incomestatement_to_df(StatementsOfIncome).name))

set()

In [102]:
df = incomestatement_to_df(StatementsOfIncome)

In [103]:
df[["startDate", "endDate"]].value_counts()

startDate   endDate   
2022-01-01  2022-06-30    44
2022-04-01  2022-06-30    44
2023-01-01  2023-06-30    44
2023-04-01  2023-06-30    44
dtype: int64